In [1]:
IN_COLAB = 'google.colab' in str(get_ipython())

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd /content/drive/MyDrive/Github/Abstract-generator/bumbleBERT/notebooks
    
    %%capture
    !pip install feedparser tokenizers transformers;

In [2]:
import os, torch, time, math, sys, re, csv
import numpy as np

sys.path.append('..' + os.sep )
from src import default

from src.data import download as dl, tokenization as tkn, custom_dataset as cd

from torch.utils.data import DataLoader
from src.model.transformer_hf import TransformerModel
from src.model.batching import CustomBatch
from src.model.generate_text import gen_some_text
from src.model.train_evaluate import train, evaluate
#from src.model.transformer import make_gpt_model # imports don't work

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Parameters

In [3]:
# ARCHITECTURE
maxLen     = 150 # maximum sentence length
vocabSize  = None # None if you want to let tokenizer do its thing
emsize     = 512 # embedding dimension
nhid       = 2048 # the dimension of the feedforward network model in torch.nn.TransformerEncoder
nlayers    = 12 # the number of torch.nn.TransformerEncoderLayer in torch.nn.TransformerEncoder
nhead      = 8 # the number of heads in the multiheadattention models
dropout    = 0.2 # the dropout value
batch_size = 10 #32
val_batch_size = 10 #32
epochs     = 10  # The number of epochs

# TOKENIZER
tknzerType = 'BPE' # type of tokenizing algorithm
trainTokenizer = True # whether to train a new tokenizer or use one already trained

TRAIN = True

### Download Dataset

In [4]:
# download data
nbrResults = 10**4 # number of data samples to download
filename = f'arxiv_{nbrResults}'
extension = '.csv'
filename += extension

filepath = default.RAW_DATA_DIR + os.sep + filename

if not os.path.exists(filepath):
    dl.arxiv_api( default.RAW_DATA_DIR, filename, max_results=nbrResults ) # TODO : CHANGE SO THAT NOT CONSTANTLY LOADING DATA
print(f'>> Using {filename} for training <<')

>> Using arxiv_10000.csv for training <<


### Format Dataset

Uses a custom dataset class, which is an iterable and callable structure that returns a sample from our dataset. Within this custom dataset, can determine all preprocessing.

In [5]:
# create dataset
dataset = cd.ArxivDataset(filepath)

### Training Tokenizer

Training of a custom tokenizer. Many options possible here, check the tokenizer training functions to try out various strategies. If he tokenizer for the dataset has already been trained, no need to run this again.

In [6]:
trainTokenizer: _ = tkn.train_custom_tokenizer(tknzerType, dataset, filename
                                            , default.TOK_DIR
                                            , vocabSize
                                            , **default.special_token_lst)

### Loading Tokenizer and Splitting Datasets

For some reason, torch tokenizers are not callable as trained. This is confusing, but c'est la vie! Instead, need to load it from file it was saved in using the PreTrainedTokenizerFast class (__call__) implemented in here. Once that's done, you can add this tokenizer as a transform to your dataset! Useful.

We also split the dataset here into training, testing and validation datasets.

In [7]:
tknzrFile = default.TOK_DIR + os.sep + filename + '_' + tknzerType + '.json'

# load PreTrainedTokenizerFast, for __call__. __call__ not implemented in
# the base Tokenizer class... that sounds silly, but it is what it is
tknzr = tkn.load_tokenizer(tknzrFile, **default.special_token_lst)

if vocabSize is None: vocabSize = tknzr.vocab_size

# set tknzr as the transform
dataset.set_transform( tknzr )

# separate dataset into train, test valid TODO : make into a function
fracTrain, fracTest, fracVal = ( 0.7, 0.2, 0.1)
trainTestVal = [ np.floor(fracTrain*len(dataset))\
                    , np.floor(fracTest*len(dataset))\
                    , len(dataset) - ( np.floor( fracTrain*len(dataset) ) +
                    np.floor( fracTest*len(dataset) ) )
                    ]

trainDataset, testDataset, valDataset =\
        torch.utils.data.random_split(dataset, [int(x) for x in trainTestVal]
                                , generator=torch.Generator().manual_seed(42) )


### Creating DataLoaders

Training is done on batches, so we need a way to extract groupings of the data in the appropriate format for our transformer model.
Note that for transformers which we are training, dataloaders outputs both src (x[:-1] and tgt ([1:]).
The collation of batches for different transformer models we have vary. For HuggingFace it's ( maxLen x batch_size ) whereas I think that the Annotated Transformer has ( batch_size x maxLen ).

NOTE : Do not use the tokenizer before the training if you use num_workers>0!
FastTokenizer does not play nicely with forking if you use it before the forking of your data:
https://stackoverflow.com/questions/62691279/how-to-disable-tokenizers-parallelism-true-false-warning

In [8]:
# create dataloaders
# uses collate function to transform batch to correct dimensions
def collate_wrapper(batch):
    return CustomBatch(batch, dim=0, maxLenModel=maxLen, padValue=tknzr.get_vocab()["<pad>"])

# dataloader for training
trainDataLoader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True
                                        , num_workers=2
                                        , collate_fn=collate_wrapper
                                        , pin_memory=True
                                        )
# dataloader for validation
valDataLoader = DataLoader(valDataset, batch_size=val_batch_size, shuffle=True
                                        , num_workers=2
                                        , collate_fn=collate_wrapper
                                        , pin_memory=True
                                        )


### Selecting model

Here we choose which model we shall use for training. For now, I've selected the black box Transformer from HuggingFace because the collate_fn I've written gives the correct input size force it... however this can easily be changed! 

In [9]:
# transformer from huggingface
# TODO : Change to the Annotated Transformer if I want
model = TransformerModel(vocabSize, emsize, nhead, nhid, nlayers, dropout).to(device)

# criterion
criterion = torch.nn.CrossEntropyLoss()#ignore_index=tknzr.get_vocab()["<pad>"])

# optimizer
# learning rate Matt used with Adam is 0.5
paramsAdam  = [{'params' : model.parameters(), 'lr' : 1e-3, 'betas' : (0.9, 0.999), 'eps' : 1e-08, 'weight_decay' : 0.0}]
paramsAdamW = [{'params' : model.parameters(), 'lr' : 5e-5, 'betas' : (0.9, 0.999), 'eps' : 1e-08, 'weight_decay' : 0.0}]
paramsSGD   = [{'params' : model.parameters(), 'lr' : 0.5, 'momentum' : 0.0, 'dampening' : 0.0, 'weight_decay' : 0.0}]

#optimizer = torch.optim.SGD( paramsSGD )
#optimizer = torch.optim.Adam( paramsAdam )
optimizer = torch.optim.AdamW( paramsAdamW )

# scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95) # 1.0 to signify no decay rate

### Training

Training loop!

In [10]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # fasttokenizer should not be used before forking. Something
                                                # to figure out. What this does is suppress some warning messages 
                                                # https://stackoverflow.com/questions/62691279/how-to-disable-tokenizers-parallelism-true-false-warning
                                                # doesn't seem to affect the timing though
if TRAIN:
    best_val_loss = float("inf")
    best_model = None
    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train( model, maxLen, trainDataLoader, device, vocabSize, epoch, optimizer, scheduler, criterion)
        val_loss = evaluate(model, maxLen, valDataLoader, len(valDataset), device, vocabSize, criterion)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
              'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                         val_loss, math.exp(val_loss)))
                                         # Why is math.exp so large????
        print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model

        scheduler.step()


    # save best model (two methods)
    modelFull = default.MODEL_DIR + os.sep + f'{filename}_epoch{epochs}.pth'
    modelWeights = default.MODEL_DIR + os.sep + f'{filename}_weights_epoch{epochs}.pth'
    modelFullBest = default.MODEL_DIR + os.sep + f'{filename}_epoch{epochs}_best.pth'
    modelWeightsBest = default.MODEL_DIR + os.sep + f'{filename}_weights_epoch{epochs}_best.pth'
    # approach 1: save model (class) entirely (uses pickle)
    torch.save(model, modelFull)
    torch.save(best_model, modelFullBest)
    # approach 2: save model weights
    torch.save(best_model.state_dict(), modelWeightsBest)

| epoch   1 |   200/  700 batches | lr 0.00 | ms/batch 53.75 | loss  6.61 | ppl   741.97
| epoch   1 |   400/  700 batches | lr 0.00 | ms/batch 52.54 | loss  5.54 | ppl   253.66
| epoch   1 |   600/  700 batches | lr 0.00 | ms/batch 52.44 | loss  5.31 | ppl   201.75
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 38.75s | valid loss 75.93 | valid ppl 946377894916161123749865122168832.00
-----------------------------------------------------------------------------------------
| epoch   2 |   200/  700 batches | lr 0.00 | ms/batch 53.87 | loss  5.06 | ppl   156.96
| epoch   2 |   400/  700 batches | lr 0.00 | ms/batch 53.43 | loss  4.96 | ppl   142.64
| epoch   2 |   600/  700 batches | lr 0.00 | ms/batch 53.45 | loss  4.87 | ppl   130.56
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 39.21s | valid loss 71.86 | valid ppl 16087822062377462293137536843

### Text Generation

Here I've simply taken the code Matt uses to generate text.

In [11]:
if not TRAIN:
    customFilename = 'arxiv_10000'
    customEpochs = 10
    modelFull = default.MODEL_DIR + os.sep + f'{customFilename}_epoch{customEpochs}_best.pth'
    modelWeights = default.MODEL_DIR + os.sep + f'{customFilename}_weights_epoch{customEpochs}_best.pth'
    
    # approach 1: load model (class) entirely (uses pickle)
    modelFullLoad = torch.load(modelFull, map_location=device)

    # approach 2: load model weights, need to have some parameter or something 
    modelLoad = TransformerModel(vocabSize, emsize, nhead, nhid, nlayers, dropout).to(device)
    modelWeightsLoad = modelLoad.load_state_dict( torch.load(modelWeights) )

In [12]:
# inspect both models
#print('model_A info...\n', modelFullLoad)
#print('\nmodel_B info...\n', modelWeightsLoad)

#print('model_A == model_B:', modelFullLoad == modelWeightsLoad)
#model = modelFullLoad
# Text generation example

#model = modelLoad
prompt = 'Where black holes change topology'
ngen = 100
decode_style = 'sample_topp' #greedy, sample_topp
model.to('cpu')
generated_text = gen_some_text(
    model, tknzr, 'cpu', maxLen, text_prompt=prompt, tokens_to_gen=ngen, vis=False,
    decode_style=decode_style)
print("Text prompt:\n", prompt)
print("Number of tokens to generate:", ngen)
print("Generated_text:\n", generated_text)

# TODO: alternative generation
# currently 'greedy method'
# see: https://huggingface.co/blog/how-to-generate

<s>  Where  black  holes  change  topology
[  122   213   121 ... 27838 20851 25253]
[0.32548374 0.47559777 0.6078751  ... 0.99993086 0.99993086 0.99993086]
topp_indices 8 [122 213 121 146 143 171 513 151]
tensor([-3.7053, -5.9343,  1.2938,  ..., -1.8073, -5.0822, -6.1341],
       grad_fn=<SelectBackward0>)
Where black holes change topology  are
<s>  Where  black  holes  change  topology  are
[  105   133   784 ... 19087 28445 27607]
[0.14281589 0.19576523 0.23616013 ... 0.99995244 0.99995244 0.99995244]
topp_indices 181 [  105   133   784  1676   122   107  2831  1358  1123   909   470   848
   588   121   151  1901  1404  1455  1849  1712  1394  1758  2268  2295
  2924  3488  5475  1883  2638  2850   474  2921  3172  1177  2829  2179
  1984  1994  2805  1059  3556  5603   681  4297   199  2875  1562  1609
  5806  1115  1629   690  1263  2758  1986  1968   986  2424  2651  1939
  2098  1160   562  3243  3308  1194  2257   175  1926  1681  3261  3926
  6793  2224  3902  1386  3645   56

[ 1627  2050  1114 ... 10083 16368 18492]
[0.5179614  0.71358204 0.7551339  ... 0.9999676  0.9999676  0.9999676 ]
topp_indices 12 [1627 2050 1114  593 2164  938 3230  134  416  773  203 2039]
tensor([-2.2180, -5.9714, -1.9282,  ..., -5.1060, -4.3746, -4.1929],
       grad_fn=<SelectBackward0>)
Where black holes change topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates
<s>  Where  black  holes  change  topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates
[  123   143   146 ...  5595 11851 25253]
[0.6400462  0.8273275  0.90405893 ... 0.99992234 0.99992234 0.99992234]
topp_indices 3 [123 143 146]
tensor([-2.5917, -6.7246,  1.7409,  ..., -2.9485, -5.0971, -6.0228],
       grad_fn=<SelectBackward0>)
Where black holes change topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .
<s>  Where  black  holes  change  topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .
[  189   340 

[  122   130   146 ... 11503 17970 16857]
[0.7125889  0.8115968  0.8548551  ... 0.99992234 0.99992234 0.99992234]
topp_indices 3 [122 130 146]
tensor([-4.3596, -5.4310,  2.5490,  ..., -3.0611, -5.5553, -4.2554],
       grad_fn=<SelectBackward0>)
Where black holes change topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .  Here ,  we  study  the  performance  of  electron  -  electron  interactions  on  the  density  of
<s>  Where  black  holes  change  topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .  Here ,  we  study  the  performance  of  electron  -  electron  interactions  on  the  density  of
[  418   107   105 ... 18744  3718 27653]
[0.6041851  0.8051888  0.86060333 ... 0.9999407  0.9999407  0.9999407 ]
topp_indices 3 [418 107 105]
tensor([-2.1180, -8.4469, -0.7944,  ..., -6.7345, -3.8126, -4.2782],
       grad_fn=<SelectBackward0>)
Where black holes change topology  are  an  important  role  in  the  Sun  in  the 

[  107   105   343 ...  8058 16304 17708]
[0.53990686 0.67544407 0.7773342  ... 0.9999357  0.9999357  0.9999357 ]
topp_indices 7 [107 105 343 134 605 210 133]
tensor([-1.4543, -7.9876, -1.1537,  ..., -6.8729, -4.8127, -4.8608],
       grad_fn=<SelectBackward0>)
Where black holes change topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .  Here ,  we  study  the  performance  of  electron  -  electron  interactions  on  the  density  of  states  in  the  metal  -  insulator  transition .  In  the  case  of  the
<s>  Where  black  holes  change  topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .  Here ,  we  study  the  performance  of  electron  -  electron  interactions  on  the  density  of  states  in  the  metal  -  insulator  transition .  In  the  case  of  the
[  134   317  1095 ... 22074  8058  1856]
[0.17340872 0.23686504 0.28096074 ... 0.99998    0.99998    0.99998   ]
topp_indices 293 [ 134  317 1095  628  321  433

[  107   105   162 ... 19132  1258  5783]
[0.51057607 0.7137595  0.7350508  ... 0.9999509  0.9999509  0.9999509 ]
topp_indices 23 [ 107  105  162  415  133  199  246  716  370  121  134  175  143  391
  323  335  321  276  355 1785  213  748  659]
tensor([-5.6155, -7.7924, -0.1815,  ..., -3.8242, -5.2802, -3.3897],
       grad_fn=<SelectBackward0>)
Where black holes change topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .  Here ,  we  study  the  performance  of  electron  -  electron  interactions  on  the  density  of  states  in  the  metal  -  insulator  transition .  In  the  case  of  the  bulk  phase  of  a  Fermi  surface  and  the
<s>  Where  black  holes  change  topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .  Here ,  we  study  the  performance  of  electron  -  electron  interactions  on  the  density  of  states  in  the  metal  -  insulator  transition .  In  the  case  of  the  bulk  phase  of  a  Fermi

...

[  107   105   133 ...  4353 21576 17708]
[0.84379816 0.89073247 0.90206236 ... 0.9999328  0.9999328  0.9999328 ]
topp_indices 2 [107 105]
tensor([-4.4961, -8.2531, -0.7015,  ..., -5.8825, -5.0277, -6.5797],
       grad_fn=<SelectBackward0>)
Where black holes change topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .  Here ,  we  study  the  performance  of  electron  -  electron  interactions  on  the  density  of  states  in  the  metal  -  insulator  transition .  In  the  case  of  the  bulk  phase  of  a  Fermi  surface  and  the  Fermi  level  is  considered .  We  observe  that  this  is  due  to  the  electron  -  phonon  interaction ,  which  is  equal  to  the
<s>  Where  black  holes  change  topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .  Here ,  we  study  the  performance  of  electron  -  electron  interactions  on  the  density  of  states  in  the  metal  -  insulator  transition .  In  the  case  of  t

[  123   146   143 ...  5595 13350 25253]
[0.46451688 0.55612594 0.63979506 ... 0.9999361  0.9999361  0.9999361 ]
topp_indices 14 [123 146 143 122 121 162 151 225 130 633 208 216 171 872]
tensor([-4.5549, -7.2691,  1.9728,  ..., -3.9518, -4.9905, -5.4984],
       grad_fn=<SelectBackward0>)
Where black holes change topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .  Here ,  we  study  the  performance  of  electron  -  electron  interactions  on  the  density  of  states  in  the  metal  -  insulator  transition .  In  the  case  of  the  bulk  phase  of  a  Fermi  surface  and  the  Fermi  level  is  considered .  We  observe  that  this  is  due  to  the  electron  -  phonon  interaction ,  which  is  equal  to  the  mechanism  of  the  lattice  energy ,
<s>  Where  black  holes  change  topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .  Here ,  we  study  the  performance  of  electron  -  electron  interactions  on  th

[  134   628   426 ...  6164 21656  5405]
[0.20990828 0.25972253 0.28509992 ... 0.99997014 0.99997014 0.99997014]
topp_indices 259 [ 134  628  426  247  317  561  276  265  465  833  210  415  539  369
  355  774  807  175 1169 1339  954  292 1645  554 1197  535 2140 1114
  614  887 1185  764 1350  321  690 1946  433 1732  334  383 1075  872
  827 1270  479  598 1642  166  121 1778  659 1156  370  960  673 1951
 1492  885 2618 2069 1731 1257  911 1514 1179 2016  740 1505 1421  955
 1742 1054  810  522  916 2456 1509 1120 1224 1953  590 1303  474  603
 1030  800  473 2184  990  209 1092 1844 1161  107 1401 1332 1398  454
  791 1200  636 1761  518 1099 1442 2515  851 2280 2321 1236  680 1599
 1670 1028 2040 1861 1095 3457 2093 1484 2129 1243 1772 2286 1307 2189
  416 1397 1639 1906 3499 2542  937 1013  552 1111 1763 1248  902  708
  536 1640 1320 2518 1626  657 2311 1831 1061 1233  816 4109 2523  392
 1540  507  687  775 1572  109 2342 1806 1545  910  545  798 1916 1741
  986 1311  567 1

[  134   454   369 ...  9280 22779  5405]
[0.08524687 0.15448675 0.1984927  ... 0.99997586 0.99997586 0.99997586]
topp_indices 302 [ 134  454  369 1642  885  265  614  247  426  415  355  465  536  690
 2311  567  986  833  276  351  561  764  292  317  794 2204 1513  539
 1339  774  321  383  854  210  628  954  433  554 1176 1442  636  598
  518 1054 1386 1156 1224 1645  334 2016 1099 1243  175  740 1951 2184
  960  990 1741 1185 1401  680  911 1270  719 2280 1505 1484  473 1114
  757  610 1075 1540  843 1248 1906 2140  775 1112 2069 1641 1727 1758
 1884  121 1398 1844  535 1233  479  827 2233 1953 1514  810  872 1626
  416 1509 1307  370 1001 1599  887 1919 1257  791  603  972 2342 1742
 1303  937 1761  687 1394  555 2618 1499 2608  538  649  545 2245  851
  916 1745 1750 2093 1107 1320  910 1200 3457 1030 1055 2193  842  800
 3845 2237 1086  590  955 1350 1161  999  807  655  392 1421 1732 2826
 1414 1095 1772 2872  899  552  166 1145  921 1904 1166  798 1263  816
 2078 1013 1131 2

[ 1224   911  1600 ... 28982  9133 28574]
[0.8631342  0.8766764  0.8813412  ... 0.99994415 0.99994415 0.99994415]
topp_indices 1 [1224]
tensor([-1.7183, -5.9348, -2.0315,  ..., -5.4532, -3.2117, -3.9878],
       grad_fn=<SelectBackward0>)
Where black holes change topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .  Here ,  we  study  the  performance  of  electron  -  electron  interactions  on  the  density  of  states  in  the  metal  -  insulator  transition .  In  the  case  of  the  bulk  phase  of  a  Fermi  surface  and  the  Fermi  level  is  considered .  We  observe  that  this  is  due  to  the  electron  -  phonon  interaction ,  which  is  equal  to  the  mechanism  of  the  lattice  energy ,  which  appears  to  change  the  electron  -  phonon  interaction .  The  resulting  results  are  in  an  important  role
<s>  Where  black  holes  change  topology  are  an  important  role  in  the  Sun  in  the  high  -  Tc  cuprates .  Here ,  we  

In [13]:
tknzr.decode([107])

' the'